In [8]:
#####################################################
###   Create Scoring Table for Auto Agent Damage  ###
#####################################################

###################
### Credentials ###
###################

import os
import sys
from pathlib import Path

filepath = input("file path to credentials: ")
sys.path.append(filepath)
from credentials import hostname, protocol, token, token_pem

In [9]:
#############################
### Connect with SAS Viya ###
#############################

import swat

access_token = open(token, "r").read()
conn =  swat.CAS(hostname=hostname, username=None, password=access_token, ssl_ca_list=token_pem, protocol=protocol)
conn

CAS('create.demo.sas.com', 443, protocol='https', name='py-session-2', session='b9f48812-cb22-f844-98c4-125c880adf75')

In [10]:
import base64
import pandas as pd

In [ ]:
image_path = ".../images/auto_damage/ai_damage/image_1023.png"
encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii') #.decode('utf-8')
with open(image_path, 'rb') as f:
    binary_image = f.read()


In [18]:
id = 11
report = """On 10/03/2025 at approximately 14:30 hours, I, Officer Laura Chen (Badge Number 501), of the Central City Traffic Division, was dispatched to a two-vehicle collision at the intersection of Elm Street and Willow Avenue. The vehicles involved were Vehicle 1, a white 2024 BMW 2 Series (License Plate RGE-113, VIN: 5N1DA45B2SCXXXXXX), driven by Sofia Rodriguez (DOB 04/22/1985, License Number S987654321, Address: 78 Oak Lane, Central City, USA). Ms. Rodriguez was the sole occupant. Vehicle 2 was a black 2018 Toyota Camry (License Plate TYY-408, VIN: 4T1BF1HK1JUXXXXXX), driven by David Kim (DOB 11/15/1976, License Number K123456789, Address: 205 Maple Street, Central City, USA), who was also the sole occupant. Based on witness statements and physical evidence at the scene (including skid marks and point of impact), Vehicle 2 (Toyota Camry), driven by David Kim, was determined to be at fault. The investigation revealed that Mr. Kim failed to stop at the red light while traveling eastbound on Willow Avenue, resulting in his vehicle striking the driver's side of the BMW, which was traveling northbound on Elm Street with the right-of-way. Damage Assessment: Vehicle 1 (BMW): Suffered major front-end damage, including a shattered front bumper, crushed grille, broken headlights, and a bent hood. The impact also caused significant body damage to the driver's side front quarter panel. Estimated damage is severe/non-drivable. Vehicle 2 (Toyota Camry): Sustained moderate damage primarily to the front passenger-side corner, including a crumpled fender, damaged headlight, and minor damage to the bumper cover. Estimated damage is moderate/drivable with restrictions. Injuries and Citations: Ms. Rodriguez (Vehicle 1) reported neck pain and whiplash symptoms. She was transported to City General Hospital via ambulance for non-life-threatening injuries. Mr. Kim (Vehicle 2) refused medical attention at the scene. Mr. Kim was issued a citation for Failure to Obey a Traffic Control Device (Running a Red Light) (Statute § 316.075). Insurance Claims: The claim process will be directed at the at-fault party's liability insurance. Ms. Rodriguez (Vehicle 1) will file a third-party liability claim against Mr. Kim's (Vehicle 2) bodily injury liability (for her medical expenses) and property damage liability (for the repairs or total loss of her BMW). Mr. Kim will use his own collision coverage to repair the damage to his Toyota Camry."""

In [19]:
### define class ###
class damage(object):
    def __init__(self, id, report, encoded_image, binary_image):
        self.id, self.report, self.encoded_image, self.binary_image = id, report, encoded_image, binary_image

data = [damage(id, report, encoded_image, binary_image)] # additional rows add to list ", damage(id2, report2, encoded_image2)", etc.
df = pd.DataFrame(i.__dict__ for i in data)

In [21]:
### upload dataframe directly to CAS ###
conn.upload_frame(df, casout=dict(caslib='casuser', name='auto_damage_score_demo', promote=True))

### Note: in order for ID to score using the encoded_image object, the variable length in ID was changed to 10485760, which may be due to the model requirements ###

NOTE: Cloud Analytic Services made the uploaded file available as table AUTO_DAMAGE_SCORE_DEMO in caslib CASUSER(Chris.Parrish@sas.com).
NOTE: The table AUTO_DAMAGE_SCORE_DEMO has been created in caslib CASUSER(Chris.Parrish@sas.com) from binary data uploaded to Cloud Analytic Services.


CASTable('AUTO_DAMAGE_SCORE_DEMO', caslib='CASUSER(Chris.Parrish@sas.com)')

In [ ]:
### altering binary_image column to varbinary(image) ###
conn.table.alterTable(
    caslib='fsbu', name='auto_damage_score_test',
    columns=[dict(name='binary_image', newType='VARBINARY', binaryType='IMAGE')]
    )

+ Elapsed: 0.000649s, user: 0.000277s, sys: 0.000135s, mem: 0.637mb

In [ ]:
### to csv and manual upload to CAS ###
df.to_csv(".../auto_accident_reports/auto_damage_score.csv")

In [3]:
conn.loadactionset('table')
conn.loadactionset('image')
caslib_images = 'images'
caslib_output = 'public'
caslib_models = 'casuser'
table_images = 'auto_claim_images'

NOTE: Added action set 'table'.
NOTE: Added action set 'image'.


In [17]:
### assign caslib to directory where images reside (set subDirectories=True if multiple directories)
### caslib is required to load images into CAS table
### images could be in Github (on SAS Server in Viya) or a path to a file system
imagepath = input("file path within viya to Github directory with images: ")

conn.table.addCaslib(
        name=caslib_images,
		description='',
        session=True,
		path=imagepath,
        subDirectories=True,
        datasource=dict(srctype="path")
)

NOTE: Failed to resolve path /create-export/create/homes/Chris.Parrish@sas.com/chris_git_sas_viya/data/images/auto_damage/real_damage/IMG_1552.JPG/ for caslib images.
ERROR: The caslib images is a duplicate, parent or subpath of caslib images.
ERROR: Could not add caslib 'images'. Make sure that the caslib does not already exist and that you have permissions to add caslibs to Cloud Analytic Services.
ERROR: The action stopped due to errors.


+ Elapsed: 0.0048s, user: 0.00101s, sys: 0.000142s, mem: 0.342mb

In [22]:
### load images into CAS table
### images are labeled based on directory names
auto_claim_images = conn.image.loadimages(
    caslib=caslib_images,
    recurse=True,
    labelLevels=-1,
    addColumns=("CHANNELCOUNT", "CHANNELTYPE", "DEPTH", "HEIGHT", "ORIENTATION", "POSITION", "SPACING", "SPAN", "WIDTH"),
    casOut=dict(caslib=caslib_output, name=table_images, replace=True)
)

NOTE: Loaded 344 images from /create-export/create/homes/Chris.Parrish@sas.com/chris_git_sas_viya/data/images/auto_damage/ into table auto_claim_images.


In [23]:
conn.simple.freq(
    table=dict(caslib=caslib_output, name=table_images, vars='_label_')
)

,Column,CharVar,FmtVar,Level,Frequency
0,_label_,ai_damage,ai_damage,1,41.0
1,_label_,no_damage,no_damage,2,170.0
2,_label_,real_damage,real_damage,3,133.0


In [24]:
conn.table.columnInfo(
    table=dict(caslib=caslib_output, name=table_images)
)

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,_channelCount_,,1,int64,8,20,,0,0
1,_channelType_,,2,varchar,3,3,,0,0
2,_depth_,,3,int64,8,20,,0,0
3,_height_,,4,int64,8,20,,0,0
4,_orientation_,,5,varbinary,32,32,,0,0
5,_position_,,6,varbinary,16,16,,0,0
6,_spacing_,,7,varbinary,16,16,,0,0
7,_span_,,8,int64,8,20,,0,0
8,_width_,,9,int64,8,20,,0,0
9,_image_,,10,varbinary(image),9463980,9463980,,0,0


In [18]:
conn.table.fetch(
    table=dict(caslib=caslib_output, name=table_images), #, vars='_path_'
    to=5
)

NOTE: Varbinary column '_orientation_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_position_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_spacing_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_image_' may not be in a format that can be displayed consistently.


,_channelCount_,_channelType_,_depth_,_height_,_orientation_,_position_,_spacing_,_span_,_width_,_image_,_size_,_path_,_label_,_type_,_id_
0,3,8U,1,1024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,1536,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,3562448,/create-export/create/homes/Chris.Parrish@sas....,ai_damage,png,1
1,3,8U,1,3024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,4032,b'\xff\xd8\xff\xe0\x00\x14JFIF\x00\x01\x01\x01...,7044014,/create-export/create/homes/Chris.Parrish@sas....,no_damage,jpg,52
2,3,8U,1,3024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,4032,b'\xff\xd8\xff\xe0\x00\x14JFIF\x00\x01\x01\x01...,4692560,/create-export/create/homes/Chris.Parrish@sas....,no_damage,jpg,107
3,3,8U,1,3024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,4032,b'\xff\xd8\xff\xe0\x00\x14JFIF\x00\x01\x01\x01...,5884748,/create-export/create/homes/Chris.Parrish@sas....,no_damage,jpg,158
4,3,8U,1,3024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,4032,b'\xff\xd8\xff\xe0\x00\x14JFIF\x00\x01\x01\x01...,3577730,/create-export/create/homes/Chris.Parrish@sas....,no_damage,jpg,204


In [25]:
### drop 'no_damage' images
conn.table.deleteRows(
    table=dict(caslib=caslib_output, name=table_images, where=("_id_ ne 1"))
)

[rowsDeleted]

 343

+ Elapsed: 0.00351s, sys: 0.0173s, mem: 2.23mb

In [26]:
conn.table.fetch(
    table=dict(caslib=caslib_output, name=table_images)
)

NOTE: Varbinary column '_orientation_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_position_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_spacing_' may not be in a format that can be displayed consistently.
NOTE: Varbinary column '_image_' may not be in a format that can be displayed consistently.


,_channelCount_,_channelType_,_depth_,_height_,_orientation_,_position_,_spacing_,_span_,_width_,_image_,_size_,_path_,_label_,_type_,_id_
0,3,8U,1,1024,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x0...,1,1536,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,3562448,/create-export/create/homes/Chris.Parrish@sas....,ai_damage,png,1


In [ ]:
### promoted table to casuser caslib; may need to delete in Viya to overwrite ###
conn.table.promote(
    caslib=caslib_output, name=table_images, targetLib=caslib_models
)

NOTE: Cloud Analytic Services promoted table AUTO_CLAIM_IMAGES in caslib Public to table auto_claim_images in caslib CASUSER(Chris.Parrish@sas.com).


+ Elapsed: 0.00996s, sys: 0.00971s, mem: 0.307mb

In [7]:
conn.table.alterTable(
    caslib=caslib_models, name=table_images,
    columns=[dict(name='_image_', newType='VARBINARY', binaryType='IMAGE')]
    )

+ Elapsed: 0.000676s, user: 0.000328s, sys: 0.000129s, mem: 0.662mb